In [1]:
import mediapipe as mp
import numpy as np
import torch
#import torch_geometric
from PIL import Image

from torch_geometric.data import Data
import cv2
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import glob
import os
import math 
from IPython.display import clear_output
import itertools
import seaborn as sb
from tqdm import tqdm


In [2]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
kernel_size = 20



model = DenseNet121(input_shape=(kernel_size*2,kernel_size*2,3),weights=None, include_top=False)

# Choose the desired low-level feature layers
low_level_layers = [
    model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
    model.get_layer('conv3_block12_concat')# Output of the last layer in the second dense block
]
# low_level_layers = [
#     model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
#     model.get_layer('conv3_block12_concat'), 
#     model.get_layer('conv4_block24_concat')# Output of the last layer in the second dense block
# ]

# Create a new model with the desired low-level feature layers as outputs
feature_extractor = tf.keras.Model(inputs=model.input, outputs=[layer.output for layer in low_level_layers])
feature_extractor.load_weights("/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.h5")
feature_extractor.trainable=False

2024-09-27 17:38:33.103761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-27 17:38:33.126644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-27 17:38:33.128148: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-27 17:38:33.131112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
feature_extractor.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 40, 40, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 46, 46, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 20, 20, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormalization)  (None, 20, 20, 64)   256         ['conv1/conv[0][0]']             
                                                                                              

 conv2_block4_1_bn (BatchNormal  (None, 10, 10, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 10, 10, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block4_2_conv (Conv2D)   (None, 10, 10, 32)   36864       ['conv2_block4_1_relu[0][0]']    
                                                                                                  
 conv2_block4_concat (Concatena  (None, 10, 10, 192)  0          ['conv2_block3_concat[0][0]',    
 te)                                                              'conv2_block4_2_conv[0][0]']    
          

                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 5, 5, 128)   512         ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 5, 5, 128)   0           ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 5, 5, 32)     36864       ['conv3_block2_1_relu[0][0]']    
                                                                                                  
 conv3_block2_concat (Concatena  (None, 5, 5, 192)   0           ['conv3_block1_concat[0][0]',    
 te)      

 conv3_block6_2_conv (Conv2D)   (None, 5, 5, 32)     36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 5, 5, 320)   0           ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_block7_0_bn (BatchNormal  (None, 5, 5, 320)   1280        ['conv3_block6_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block7_0_relu (Activatio  (None, 5, 5, 320)   0           ['conv3_block7_0_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block11_0_relu (Activati  (None, 5, 5, 448)   0           ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 5, 5, 128)    57344       ['conv3_block11_0_relu[0][0]']   
                                                                                                  
 conv3_block11_1_bn (BatchNorma  (None, 5, 5, 128)   512         ['conv3_block11_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_1_relu (Activati  (None, 5, 5, 128)   0           ['conv3_block11_1_bn[0][0]']     
 on)      

In [8]:

#input_image = tf.random.uniform((10,40,40,3))
def DenseNet_feature_extractor(input_image) :
    #input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    #print("input shape", input_image.shape)
    input_image = tf.cast(input_image, tf.float32)
    input_image = tf.keras.applications.densenet.preprocess_input(input_image)
    #input_image = tf.expand_dims(input_image, axis=0)

    feature_extracted = feature_extractor.predict(input_image,verbose=False)
    #print(feature_extracted[0].shape,feature_extracted[1].shape)
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[0])
    flatten = tf.keras.layers.Flatten()(global_pooling)
    total_feature =flatten
    for layer_cnt in range(1,2) : 
        global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[layer_cnt])
        flatten = tf.keras.layers.Flatten()(global_pooling)
        total_feature = tf.concat([flatten,total_feature],1)
    return total_feature

In [4]:
DenseNet_feature_extractor(tf.random.uniform((1,32,32,3)))

2024-09-11 20:51:15.117972: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-09-11 20:51:15.986352: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-11 20:51:15.986875: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-11 20:51:15.986897: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-09-11 20:51:15.987667: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-11 20:51:15.987722: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


<tf.Tensor: shape=(1, 640), dtype=float32, numpy=
array([[-7.91183352e-01,  1.81435883e-01,  5.12061305e-02,
         1.51571959e-01,  2.73062617e-01,  2.20516473e-01,
         8.50040615e-02, -3.21160071e-02, -1.00141071e-01,
         1.69137478e-01,  9.53952223e-03, -1.32887527e-01,
         6.50334880e-02,  2.54359748e-02,  1.08470701e-01,
        -1.09451473e-01, -1.68026671e-01, -1.41208082e-01,
         5.45352176e-02, -1.70157194e-01,  2.34940171e-01,
         2.69435614e-01, -4.33770241e-03,  2.61591673e-02,
        -1.29319638e-01,  1.52293950e-01, -1.96206599e-01,
        -1.84592903e-01,  1.30117312e-01,  7.66088516e-02,
         6.67965561e-02, -1.40979603e-01,  1.61852658e-01,
        -6.16849549e-02,  1.02752127e-01,  9.14865732e-03,
         2.54988652e-02, -4.72372733e-02,  2.20003605e-01,
         4.35844064e-02, -3.87201309e-02,  1.35180935e-01,
         5.58026582e-02,  6.24378063e-02,  1.34986207e-01,
        -2.05186233e-01,  2.86504142e-02,  8.97623375e-02,
      

In [9]:
kernel_size= 20
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh

connections = mp_face_mesh.FACEMESH_TESSELATION
edges = []
for connection in connections:
    edge = [connection[0], connection[1]]
    edges.append(edge)
    edge  =[connection[1], connection[0]]
    edges.append(edge)
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [11]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped*.jpg")

In [12]:
len(paths)

136334

In [34]:
for path in tqdm(paths, desc="Processing images"):   
        #print(path)
        NAME_LIST = path.split("/")[-2].split("_")
        #print(NAME_LIST)
        #print("***********************************")

        print(path)
        print(path.split("/")[-2].split("_")[2])


Processing images:   9%|█▍              | 1390/15549 [00:00<00:01, 13897.56it/s]

/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0040.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0125.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0210.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0045.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0050.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client048_laptop_SD_ipad_video_scene01/cropped_0055.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFS

Processing images:  29%|████▋           | 4497/15549 [00:00<00:00, 14927.58it/s]

spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0160.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0165.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0170.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0175.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0180.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client026_android_SD_ipad_video_scene01/cropped_0185.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSF

Processing images:  49%|███████▊        | 7589/15549 [00:00<00:00, 15139.60it/s]

...

/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0290.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0295.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0015.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0020.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0025.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client023_android_SD_iphone_video_scene01/cropped_0030.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/M

Processing images:  90%|█████████████▍ | 13987/15549 [00:00<00:00, 15173.34it/s]

spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0220.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0045.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0050.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0055.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0060.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_spoof_attack_client036_laptop_SD_iphone_video_scene01/cropped_0065.jpg
spoof
/media/hassan-hossein/B660FEE360FEA8EF/M

Processing images: 100%|███████████████| 15549/15549 [00:01<00:00, 15033.56it/s]

live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0135.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0140.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0145.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0150.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0155.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0160.jpg
live
/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/replay_test_live_real_client032_laptop_SD_scene01/cropped_0165.jpg
live
/

In [7]:
paths

['/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0000.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0005.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0010.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0015.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0020.jpg',
 '/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/replay_dev_live_real_client003_session01_webcam_authenticate_adverse_1/cropped_0025.jpg',
 '/media/hassan-hossei

In [13]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/'+os.path.basename(path)+'.pt')


Processing images:   0%|                             | 0/136334 [00:00<?, ?it/s]2024-09-18 08:11:04.318893: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-09-18 08:11:04.858021: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-18 08:11:04.858512: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-18 08:11:04.858525: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-09-18 08:11:04.859126: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-18 08:11:04.859168: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be o

In [18]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/16'+os.path.basename(path)+'.pt')


Processing images: 100%|██████████████████| 22296/22296 [42:25<00:00,  8.76it/s]


In [7]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for qulity in [20,30,50,90] :
                img_arg = tf.image.adjust_jpeg_quality(img, qulity).numpy()
                #print(img.shape)

                landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

                list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
                #print(list_of_points_face)

                list_of_points_face = np.clip(list_of_points_face,0,256)

                list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

                img_arg = np.pad(img_arg, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                         mode='constant', constant_values=0) 


                patches= []

                list_of_points_face= np.round(list_of_points_face).astype(int)

                for (x1, y1) in list_of_points_face:

                     p_img = img_arg[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

                     patches.append(p_img)  

                patches = np.array(patches,dtype =np.float32)
                features = DenseNet_feature_extractor(patches) 
                #print(patches.shape)
                #print(patches.dtype)
                vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

                data = Data(x=vertices, edge_index=edges,y =label)
                torch.save(data, os.path.dirname(path)+'/argument_'+str(qulity)+"_"+os.path.basename(path)+'.pt')


Processing images:   0%|                              | 0/22296 [00:00<?, ?it/s]2024-07-01 16:03:22.954148: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-07-01 16:03:23.815144: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 16:03:23.815654: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 16:03:23.815668: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-07-01 16:03:23.816526: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 16:03:23.816598: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be o

In [5]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped*.jpg")

In [11]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for qulity in [10,20,30,50,90] :
                
                if os.path.exists(os.path.dirname(path)+'/argument_'+str(qulity)+"_"+os.path.basename(path)+'.pt'):
                    continue
                img_arg = tf.image.adjust_jpeg_quality(img, qulity).numpy()
                #print(img.shape)

                landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

                list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
                #print(list_of_points_face)

                list_of_points_face = np.clip(list_of_points_face,0,256)

                list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

                img_arg = np.pad(img_arg, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                         mode='constant', constant_values=0) 


                patches= []

                list_of_points_face= np.round(list_of_points_face).astype(int)

                for (x1, y1) in list_of_points_face:

                     p_img = img_arg[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

                     patches.append(p_img)  

                patches = np.array(patches,dtype =np.float32)
                features = DenseNet_feature_extractor(patches) 
                #print(patches.shape)
                #print(patches.dtype)
                vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

                data = Data(x=vertices, edge_index=edges,y =label)
                torch.save(data, os.path.dirname(path)+'/argument_'+str(qulity)+"_"+os.path.basename(path)+'.pt')


Processing images: 100%|██████████████| 136334/136334 [2:26:08<00:00, 15.55it/s]


In [12]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/*/cropped*.jpg")

In [13]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for qulity in [20,30,50,90] :
                
                if os.path.exists(os.path.dirname(path)+'/argument_'+str(qulity)+"_"+os.path.basename(path)+'.pt'):
                    continue
                img_arg = tf.image.adjust_jpeg_quality(img, qulity).numpy()
                #print(img.shape)

                landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

                list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
                #print(list_of_points_face)

                list_of_points_face = np.clip(list_of_points_face,0,256)

                list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

                img_arg = np.pad(img_arg, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                         mode='constant', constant_values=0) 


                patches= []

                list_of_points_face= np.round(list_of_points_face).astype(int)

                for (x1, y1) in list_of_points_face:

                     p_img = img_arg[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

                     patches.append(p_img)  

                patches = np.array(patches,dtype =np.float32)
                features = DenseNet_feature_extractor(patches) 
                #print(patches.shape)
                #print(patches.dtype)
                vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

                data = Data(x=vertices, edge_index=edges,y =label)
                torch.save(data, os.path.dirname(path)+'/argument_'+str(qulity)+"_"+os.path.basename(path)+'.pt')


Processing images: 100%|████████████████| 15549/15549 [2:10:02<00:00,  1.99it/s]


In [ ]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/*/cropped*.jpg")